In [1]:
import argparse
import isce
import isceobj
import os
from osgeo import gdal
from osgeo import gdal_array
import numpy as np
import xml.etree.ElementTree as ET



In [2]:
###Folowing files need to be the same size as your output grid.
latfile  = '/data/krd86/cropped/geom_reference/lat.rdr.4alks_20rlks.full.vrt'
lonfile  = '/data/krd86/cropped/geom_reference/lon.rdr.4alks_20rlks.full.vrt'

In [8]:
###Files to resample from geo coords
infile       = 'geo.int'
resampMethod = 'near'
outfile      = 'ungeo.int'

In [41]:
def writeVRT(infile, latFile, lonFile):
#This function is modified from isce2gis.py
            tree = ET.parse(infile + '.vrt')
            root = tree.getroot()

            meta = ET.SubElement(root, 'metadata')
            meta.attrib['domain'] = "GEOLOCATION"
            meta.tail = '\n'
            meta.text = '\n    '


            rdict = { 'Y_DATASET' : latFile,
                      'X_DATASET' : lonFile,
                      'X_BAND' : "1",
                      'Y_BAND' : "1",
                      'PIXEL_OFFSET': "0",
                      'LINE_OFFSET' : "0",
                      'LINE_STEP' : "1",
                      'PIXEL_STEP' : "1" }

            for key, val in rdict.items():
                data = ET.SubElement(meta, 'mdi')
                data.text = val
                data.attrib['key'] = key
                data.tail = '\n    '

            data.tail = '\n'
            tree.write(infile + '.vrt')

In [34]:
##get number of rows and columns in radar coords file (lat)
raster = gdal.Open(latfile)
radarnx=raster.RasterXSize
radarny=raster.RasterYSize
cols,rows=np.meshgrid(np.arange(1,radarnx+1),np.arange(radarny,0,-1))

#make new files of columns and rows, in radar coordinates
colfile = 'cols.r4'
rowfile = 'rows.r4'
gcolfile = 'geo_cols.r4'
growfile = 'geo_rows.r4'
driver=gdal.GetDriverByName('ISCE')
colds = driver.Create(colfile,radarnx,radarny,1,gdal.GDT_Float32)
colds.GetRasterBand(1).WriteArray(cols)
colds=None

colds = driver.Create(rowfile,radarnx,radarny,1,gdal.GDT_Float32)
colds.GetRasterBand(1).WriteArray(rows)
colds=None

cmd = 'fixImageXml.py -i '+colfile+' -f'
os.system(cmd)
cmd = 'fixImageXml.py -i '+rowfile+' -f'
os.system(cmd)
writeVRT(colfile, latfile, lonfile)
writeVRT(rowfile, latfile, lonfile)

In [35]:
##get number of rows and columns + min/max range of input file
raster = gdal.Open(infile)
geonx=raster.RasterXSize
geony=raster.RasterYSize

minLon = raster.GetGeoTransform()[0]
deltaLon = raster.GetGeoTransform()[1]
maxLat = raster.GetGeoTransform()[3]
deltaLat = raster.GetGeoTransform()[5]
minLat = maxLat + geony*deltaLat
maxLon = minLon + geonx*deltaLon
WSEN = str(minLon)+' '+str(minLat)+' '+str(maxLon)+' '+str(maxLat)


In [38]:
#geocode row and col file
cmd = 'gdalwarp -of ISCE -geoloc  -te '+WSEN+' -tr '+str(deltaLon)+' '+str(deltaLat)+' -srcnodata 0 -dstnodata 0  -wt float32 -r bilinear ' +rowfile +'.vrt ' + growfile
print(cmd)
os.system(cmd)
cmd = 'gdalwarp -of ISCE -geoloc  -te '+WSEN+' -tr '+str(deltaLon)+' '+str(deltaLat)+' -srcnodata 0 -dstnodata 0  -wt float32 -r bilinear ' +colfile +'.vrt ' + gcolfile
print(cmd)
os.system(cmd)
cmd = 'fixImageXml.py -i '+gcolfile+' -f'
os.system(cmd)
cmd = 'fixImageXml.py -i '+growfile+' -f'
os.system(cmd)

gdalwarp -of ISCE -geoloc  -te -119.7 35.8 -119.5 36.0 -tr 0.0001 -0.0001 -srcnodata 0 -dstnodata 0  -wt float32 -r bilinear rows.r4.vrt geo_rows.r4
gdalwarp -of ISCE -geoloc  -te -119.7 35.8 -119.5 36.0 -tr 0.0001 -0.0001 -srcnodata 0 -dstnodata 0  -wt float32 -r bilinear cols.r4.vrt geo_cols.r4


0

In [46]:
###set any geocoding info
WSEN = '0.5 0.5 '+str(radarnx+0.5)+' '+str(radarny+0.5)
writeVRT(infile, growfile, gcolfile)
print('this will just append geoloc info to the vrt - perhaps check to make sure multiple copies are not ending up there')

In [47]:
cmd = 'gdalwarp -of ISCE -geoloc  -te '+ WSEN + ' -tr 1 1 -srcnodata 0 -dstnodata 0 ' + ' -wt cfloat32 -r ' + resampMethod + ' ' + infile +'.vrt ' outfile
print (cmd)
os.system(cmd)
cmd = 'fixImageXml.py -i '+outfiles+' -f'
os.system(cmd)

gdalwarp -of ISCE -geoloc  -te 0.5 0.5 1500.5 875.5 -tr 1 1 -srcnodata 0 -dstnodata 0  -wt cfloat32 -r near geo.int.vrt hithere.int


0